In [1]:
import datetime
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import random
from operator import itemgetter
import time
import copy
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

random.seed(26)


In [2]:
df_train = pd.read_csv("act_train.csv", parse_dates=['date'])

In [3]:
df_train.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [4]:
df_ppl = pd.read_csv('people.csv', parse_dates=['date'])

In [5]:
df_ppl.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [6]:
df_test = pd.read_csv('act_test.csv', parse_dates=['date'])

In [7]:
df_test.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10
0,ppl_100004,act1_249281,2022-07-20,type 1,type 5,type 10,type 5,type 1,type 6,type 1,type 1,type 7,type 4,NaN
1,ppl_100004,act2_230855,2022-07-20,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 682
2,ppl_10001,act1_240724,2022-10-14,type 1,type 12,type 1,type 5,type 4,type 6,type 1,type 1,type 13,type 10,NaN
3,ppl_10001,act1_83552,2022-11-27,type 1,type 20,type 10,type 5,type 4,type 6,type 1,type 1,type 5,type 5,NaN
4,ppl_10001,act2_1043301,2022-10-15,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 3015


In [8]:
df_sample_submission = pd.read_csv('sample_submission.csv')

In [9]:
df_sample_submission.head()

,activity_id,outcome
0,act1_1,0
1,act1_100006,0
2,act1_100050,0
3,act1_100065,0
4,act1_100068,0


In [10]:
df_train_merge = pd.merge(df_train, df_ppl, on='people_id')
df_test_merge = pd.merge(df_test, df_ppl, on='people_id')

In [11]:
df_train.columns

Index(['people_id', 'activity_id', 'date', 'activity_category', 'char_1',
       'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8',
       'char_9', 'char_10', 'outcome'],
      dtype='object')

In [12]:
df_test_merge.columns

Index(['people_id', 'activity_id', 'date_x', 'activity_category', 'char_1_x',
       'char_2_x', 'char_3_x', 'char_4_x', 'char_5_x', 'char_6_x', 'char_7_x',
       'char_8_x', 'char_9_x', 'char_10_x', 'char_1_y', 'group_1', 'char_2_y',
       'date_y', 'char_3_y', 'char_4_y', 'char_5_y', 'char_6_y', 'char_7_y',
       'char_8_y', 'char_9_y', 'char_10_y', 'char_11', 'char_12', 'char_13',
       'char_14', 'char_15', 'char_16', 'char_17', 'char_18', 'char_19',
       'char_20', 'char_21', 'char_22', 'char_23', 'char_24', 'char_25',
       'char_26', 'char_27', 'char_28', 'char_29', 'char_30', 'char_31',
       'char_32', 'char_33', 'char_34', 'char_35', 'char_36', 'char_37',
       'char_38'],
      dtype='object')

In [13]:
df_train.shape

(2197291, 15)

In [14]:
df_train_merge.shape

(2197291, 55)

In [15]:
df_ppl.shape

(189118, 41)

In [16]:
df_test.shape

(498687, 14)

In [17]:
df_test_merge.shape

(498687, 54)

In [18]:
df_train_merge.to_csv('train_merged.csv', index = False)

In [19]:
df_test_merge.to_csv('test_merged.csv', index = False)

In [20]:
people = pd.read_csv("people.csv",
                     dtype={'people_id': np.str,
                            'activity_id': np.str,
                            'char_38': np.int32},
                     parse_dates=['date'])

In [21]:
train = pd.read_csv("act_train.csv",
                    dtype={'people_id': np.str,
                           'activity_id': np.str,
                           'outcome': np.int8},
                    parse_dates=['date'])

In [22]:
test = pd.read_csv("act_test.csv",
                   dtype={'people_id': np.str,
                          'activity_id': np.str},
                   parse_dates=['date'])

In [23]:
for table in [train, test]:
    table['year'] = table['date'].dt.year
    table['month'] = table['date'].dt.month
    table['day'] = table['date'].dt.day
    table.drop('date', axis=1, inplace=True)
    table['activity_category'] = table['activity_category'].str.lstrip('type ').astype(np.int32)
    for i in range(1, 11):
        table['char_' + str(i)].fillna('type -99', inplace=True)
        table['char_' + str(i)] = table['char_' + str(i)].str.lstrip('type ').astype(np.int32)




In [24]:
people['year'] = people['date'].dt.year
people['month'] = people['date'].dt.month
people['day'] = people['date'].dt.day
people.drop('date', axis=1, inplace=True)
people['group_1'] = people['group_1'].str.lstrip('group ').astype(np.int32)

In [25]:
for i in range(1, 10):
    people['char_' + str(i)] = people['char_' + str(i)].str.lstrip('type ').astype(np.int32)
for i in range(10, 38):
    people['char_' + str(i)] = people['char_' + str(i)].astype(np.int32)



In [26]:
train = pd.merge(train, people, how='left', on='people_id', left_index=True)
train.fillna(-99, inplace=True)

test = pd.merge(test, people, how='left', on='people_id', left_index=True)
test.fillna(-99, inplace=True)



In [27]:
train.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,year_y,month_y,day_y
0,ppl_100,act2_1734928,4,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
0,ppl_100,act2_2434093,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
0,ppl_100,act2_3404049,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
0,ppl_100,act2_3651215,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
0,ppl_100,act2_4109017,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29


In [28]:
test.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,year_y,month_y,day_y
3,ppl_100004,act1_249281,1,5,10,5,1,6,1,1,...,1,1,1,1,1,1,76,2022,7,20
3,ppl_100004,act2_230855,5,-99,-99,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,76,2022,7,20
5,ppl_10001,act1_240724,1,12,1,5,4,6,1,1,...,1,1,1,1,1,1,90,2022,10,14
5,ppl_10001,act1_83552,1,20,10,5,4,6,1,1,...,1,1,1,1,1,1,90,2022,10,14
5,ppl_10001,act2_1043301,5,-99,-99,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,90,2022,10,14


In [45]:
train.to_csv('train_converted.csv', index = False)

In [46]:
test.to_csv('test_converted.csv', index = False)

In [49]:
train = pd.read_csv('train_converted.csv')

In [50]:
test = pd.read_csv('test_converted.csv')

In [51]:
train.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,year_y,month_y,day_y
0,ppl_100,act2_1734928,4,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
1,ppl_100,act2_2434093,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
2,ppl_100,act2_3404049,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
3,ppl_100,act2_3651215,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29
4,ppl_100,act2_4109017,2,-99,-99,-99,-99,-99,-99,-99,...,0,0,1,1,1,0,36,2021,6,29


In [52]:
test.head()

,people_id,activity_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,...,char_32,char_33,char_34,char_35,char_36,char_37,char_38,year_y,month_y,day_y
0,ppl_100004,act1_249281,1,5,10,5,1,6,1,1,...,1,1,1,1,1,1,76,2022,7,20
1,ppl_100004,act2_230855,5,-99,-99,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,76,2022,7,20
2,ppl_10001,act1_240724,1,12,1,5,4,6,1,1,...,1,1,1,1,1,1,90,2022,10,14
3,ppl_10001,act1_83552,1,20,10,5,4,6,1,1,...,1,1,1,1,1,1,90,2022,10,14
4,ppl_10001,act2_1043301,5,-99,-99,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,90,2022,10,14
